In [1]:
from syft.core.hooks import TorchHook
from syft.core.workers import VirtualWorker
import torch
import torch.nn as nn
from torch.autograd import Variable as Var
import torch.optim as optim
# this is our hook
hook = TorchHook()
me = hook.local_worker
bob = VirtualWorker(id=1,hook=hook)
alice = VirtualWorker(id=2,hook=hook)

me.add_worker(bob)
me.add_worker(alice)

Hooking into Torch...
Overloading complete.


In [2]:
# create our dataset
data = Var(torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = Var(torch.FloatTensor([[0],[0],[1],[1]]))

In [3]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

In [4]:
# create our model
model = nn.Linear(2,1)

# creating our gradient objects
pred = model(data)
loss = ((pred - target)**2).sum()
loss.backward()

In [5]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [6]:
# send model to Bob
model.weight.send(bob)
model.weight.get()
# send model to Bob
model.weight.send(bob)

> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-

Parameter containing:[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker id:1>]]

In [7]:
model.weight.get()

> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/workers.py(624)register_object()
-> self.register_object(worker=worker,
793796063


Parameter containing:
-0.2842  0.2672
[torch.FloatTensor of size 1x2]

In [8]:

model.bias.send(bob)

Parameter containing:[torch.FloatTensor - Locations:[<syft.core.workers.VirtualWorker id:1>]]

In [9]:

model.bias.get()

> /Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/core/hooks.py(706)get_()
-> _id = hook_self.local_worker.id  # for brevity


AttributeError: 'Variable' object has no attribute 'id'

In [19]:



# update the model
model.zero_grad()
pred = model(data_bob)
loss = ((pred - target_bob)**2).sum()
loss.backward()
opt.step()

print(loss.get().data[0])

AttributeError: 'Variable' object has no attribute 'id'

Parameter containing:
1.00000e-02 *
 -2.4713
[torch.FloatTensor of size 1]